In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.7.1+dfsg-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.7.1+dfsg-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-2_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-2) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack ...

Vogliamo modellare il problema del cammino minimo da *1* a *n* dato un insieme di vertici su grafo  **orientato** $G=(V,A)$
Modello ILP:

$\min \sum_{a \in A} c_a x_a$

$  sum_{a \in \delta^+(1)} x_a =1 $

$  sum_{a \in \delta^{-}(n)} x_a =1 $

$  sum_{a \in \delta^{+}(i)} x_a - sum_{a \in \delta^-(i)} x_a =0~~~ i \in V \setminus\{1,n\}$

$ x_j \in \{0,1\} ~~~~~ a \in A$

In [ ]:
# mount del file che contiene i dati

from google.colab import drive
drive.mount("/content/gdrive/",force_remount=True)
#verifico cosa c'e' in "My Drive"
!ls /content/gdrive/"My Drive"/


 

Mounted at /content/gdrive/
'Colab Notebooks'   grafo_pesato.txt   grafo.txt


chiamo io *input* il file che ho aperto in lettura

In [ ]:
input = open("/content/gdrive/My Drive/grafo_pesato.txt", "r")

*read_string* è il nome che uso per memorizzare la stringa letta; *readline()* è il metodo che uso per leggere una riga dal *input*

In [ ]:
read_string = input.readline()
lista = read_string.split()  #split() mi separa i termini della stringa read_string
n = int(lista[0])
m = int(lista[1])
print("n=",n)
print("m=",m)


n= 5
m= 5


In [ ]:
archi_letti = []
costi = []
for i in range(m):   #range genera tutti i valori da 0 a m (escluso)
  read_string = input.readline()
  print("\n",read_string)
  lista = read_string.split()
  i = int(lista[0])
  j = int(lista[1])
  c = int(lista[2])
  archi_letti += [[i,j]] #aggiungo [i,j] appena letti alla lista dei lati
  costi += [c]
  print("letto arco", i,"-",j)
print(archi_letti)
print(costi)


 1 2 3

letto arco 1 - 2

 2 4 2

letto arco 2 - 4

 3 4 10

letto arco 3 - 4

 3 5 11

letto arco 3 - 5

 4 5 7
letto arco 4 - 5
[[1, 2], [2, 4], [3, 4], [3, 5], [4, 5]]
[3, 2, 10, 11, 7]


In [ ]:

archi=[]
for i in range(m):
  archi += ["a"+str(archi_letti[i][0])+str(archi_letti[i][1])]
print(archi)



['a12', 'a24', 'a34', 'a35', 'a45']


In [ ]:
COSTI = dict(zip(archi,costi))
print(COSTI)

{'a12': 3, 'a24': 2, 'a34': 10, 'a35': 11, 'a45': 7}


In [ ]:
COPPIE = dict(zip(archi,archi_letti))
print(COPPIE)

{'a12': [1, 2], 'a24': [2, 4], 'a34': [3, 4], 'a35': [3, 5], 'a45': [4, 5]}


In [ ]:
VERT = dict ()
vertici=[]
for i in range(n):
  vertici += ["vertex"+str(i+1)]
  VERT["vertex"+str(i+1)] = i+1
print(vertici)
print(VERT)

['vertex1', 'vertex2', 'vertex3', 'vertex4', 'vertex5']
{'vertex1': 1, 'vertex2': 2, 'vertex3': 3, 'vertex4': 4, 'vertex5': 5}


importiamo la libreria PYOMO

In [ ]:
from pyomo.environ import *

Creiamo modello *concrete*

In [ ]:
SSP = ConcreteModel()  # chiamo il modello SSP


 


Definiamo delle variabili binarie

In [ ]:

SSP.x = Var(archi, within=Binary)


Definiamo la funzione obiettivo ma per farlo definiamo una funzione che prende come argomento il modello (chiamato genericamente *mod*) che poi passiamo alla funzione obiettivo come *rule*

In [ ]:
def obj_rule(mod):
    return sum(COSTI[a]*mod.x[a] for a in archi)
SSP.obj = Objective(rule=obj_rule, sense = minimize )





Allo stesso modo definiamo i vincoli. Innanzitutto definiamo una funzione che prende come argomento il modello (*mod*) e un parametro i (vertice). 
Scriviamo un vincolo per ogni vertice $i \in vertici$.



In [ ]:


def degree_rule(mod, i):
    v=VERT[i]
    if v==1:
      return sum(mod.x[a] for a in archi if COPPIE[a][0]==v) == 1
    if v==n:
      return sum(mod.x[a] for a in archi if COPPIE[a][1]==v) == 1
    else:
      return sum(mod.x[a] for a in archi if COPPIE[a][0]==v) - sum(mod.x[a] for a in archi if COPPIE[a][1]==v)== 0

SSP.degree_const = Constraint(vertici, rule=degree_rule)

In [ ]:
SolverFactory('glpk').solve(SSP, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpirmn2eqf.glpk.raw --wglp /tmp/tmp_tudk0y5.glpk.glp --cpxlp
 /tmp/tmpo_54a76o.pyomo.lp
Reading problem data from '/tmp/tmpo_54a76o.pyomo.lp'...
/tmp/tmpo_54a76o.pyomo.lp:48: warning: lower bound of variable 'x1' redefined
/tmp/tmpo_54a76o.pyomo.lp:48: warning: upper bound of variable 'x1' redefined
6 rows, 6 columns, 11 non-zeros
5 integer variables, all of which are binary
53 lines were read
Writing problem data to '/tmp/tmp_tudk0y5.glpk.glp'...
35 lines were written
GLPK Integer Optimizer, v4.65
6 rows, 6 columns, 11 non-zeros
5 integer variables, all of which are binary
Preprocessing...
Objective value =   1.200000000e+01
INTEGER OPTIMAL SOLUTION FOUND BY MIP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (39693 bytes)
Writing MIP solution to '/tmp/tmpirmn2eqf.glpk.raw'...
21 lines were written


{'Problem': [{'Name': 'unknown', 'Lower bound': 12.0, 'Upper bound': 12.0, 'Number of objectives': 1, 'Number of constraints': 6, 'Number of variables': 6, 'Number of nonzeros': 11, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.017414331436157227}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
SSP.pprint()

2 Set Declarations
    degree_const_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'vertex1', 'vertex2', 'vertex3', 'vertex4', 'vertex5'}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {'a12', 'a24', 'a34', 'a35', 'a45'}

1 Var Declarations
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        a12 :     0 :   1.0 :     1 : False : False : Binary
        a24 :     0 :   1.0 :     1 : False : False : Binary
        a34 :     0 :   0.0 :     1 : False : False : Binary
        a35 :     0 :   0.0 :     1 : False : False : Binary
        a45 :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 3*x[a12] + 2*x[a24] + 10*x[a34] + 11*